# Búsqueda por probabilidad

Retomando la idea de los algoritmos de búsqueda, podemos definir un algoritmo de búsqueda que tome como función de costo la probabilidad de la transición. De esta forma, las acciones más probables dado el estado actual serán aquellas que serán elegida.

El algoritmo encontrará el camino desde el estado inicial hasta el estado final más probable. Podemos pensar esto como una heurística, donde la heurística está basada en la probabilidad.

#### Funciones iniciales

In [1]:
import numpy as np
from copy import deepcopy

Definimos una función que nos dará las coordenadas en que se encuentra el elemento vacío 'e' en la tabla. La posición se puede pensar como las coordenadas de una cuadrícula de $3 \times 3$.

In [2]:
def get_location(state, input_element):
    """
    Función para localizar la posición de un elemento del puzzle
    dentro del cuadrado o tabla.

    Arguments
    --------
    state : array
        Tabla que representa el estado del puzzle
    input_element : str
        Elemento del que se quiere obtener su localización

    Returns
    -------
        Posición en la tabla del elemento.
    """
    #Renglones
    for i, row in enumerate(state):
        #Columnas
        for j, item in enumerate(row):
            #Revisa las entradas de la tabla
            #hasta encontrar el elemento
            if item == input_element:
                #Regresa el lugar del elemento
                return i, j

## Juego del 8

Ahora definimos el problema del puzzle. Se trata de una casilla de $3\times 3$ ocupada por números de 1 a 8. Existe un hueco, que denotamos como 'e', y los números se pueden mover hacia esta casilla.

El objetivo es, a partir de una configuración inicial de las casillas, llegar a ordenarlas. En este caso, la configuración final a la que queremos llegar es:

$$[1,2,3] \\ [4,5,6] \\ [7,8,e]$$

Dentro de esta clase, definimos una función para las <b>acciones</b>; estas acciones corresponde a los números que se pueden mover hacia el espacio vacío.

La función de <b>resultado de una acción</b> regresa el resultado de aplicar esa acción (mover ese número) dado el estado actual del puzzle.

In [3]:
#Estado final del puzzle
goal = [['1', '2', '3'],
        ['4','5','6'], 
        ['7', '8', 'e']]

class Puzzle(object):
    def __init__(self, initial):
        #Estado inicial
        self.initial = initial
        #Estado actual
        self.state = initial
        #Estado meta
        self.goal = goal

    def actions(self):
        """
        Función que regresa las posibles acciones que se pueden hacer,
        según el estado actual. Por cada estado regresa tres posibles acciones.

        Returns
        -------
        actions : list
            Lista de posibles acciones/movimientos del puzzle a partir del estado actual del puzzle.
        """
        #Localiza el espacio vacío 'e' en la tabla
        row_empty, col_empty = get_location(self.state, 'e')
        #Guarda las acciones
        actions = []
        #Si no está en la primera entrada del renglón
        #Se desplaza a la izquierda
        if row_empty > 0:
            actions.append(self.state[row_empty-1][col_empty])
        #Si no está en la última entrada del renglón
        #se desplaza a la derecha
        if row_empty < 2:
            actions.append(self.state[row_empty+1][col_empty])
        #Si no está en la primera entrada de la columna
        #Se desplaza hacia arriba
        if col_empty > 0:
            actions.append(self.state[row_empty][col_empty-1])
        #Si no está en la última entrada de la columna
        #Se desplaza hacia abajo
        if col_empty < 2:
            actions.append(self.state[row_empty][col_empty+1])

        return actions

    # Return the resulting state after moving a piece to the empty space
    def result(self, action):
        """
        Obtiene el resultado a partir a partir del estado actual del puzzle y de una acción a realizar.

        Arguments
        ---------
        action : str
            Elemento que se va a deslizar en la tabla (o hacia donde el elmento 'e' se desliza)

        Returns
        -------
        rows : array
            Tabla que representa el nuevo estado del puzzle al realizar la acción.
        """
        #Nuevo estado con acción
        state = deepcopy(self.state)
        #Posición del elemento 'e'
        row_empty, col_empty = get_location(self.state, 'e')
        #Posición nueva ante la acción
        row_new, col_new = get_location(self.state, action)
        #Creación del nuevo estado del puzzle
        state[row_empty][col_empty], state[row_new][col_new] = state[row_new][col_new], state[row_empty][col_empty]

        return state

Podemos ver algunos ejemplos de cómo funciona el puzzle y como es que las acciones afectan al estado del problema de búsqueda.

In [4]:
estado = [['1','e','2'],
           ['6','3','4'],
           ['7','5','8']]

ex_puzzle = Puzzle(estado)
for a in ex_puzzle.actions():
    print(a, ex_puzzle.result(a))

3 [['1', '3', '2'], ['6', 'e', '4'], ['7', '5', '8']]
1 [['e', '1', '2'], ['6', '3', '4'], ['7', '5', '8']]
2 [['1', '2', 'e'], ['6', '3', '4'], ['7', '5', '8']]


## Definición de probabilidad

Usamos una función de probabilidad basada en métricas, lo que define una probabilidad radial. Los elementos más cercanos a la meta son los elementos más probables. 

$$p(x) = \exp\{-d(x,y)\}$$

$d$ es una métrica y $y$ será el nodo meta. Es decir, la probabilidad de un elemento se basará en su distancia al nodo meta.

In [5]:
#Guarda posiciones del estado meta
goal_positions = {}
for number in ['1','2','3','4','5','6','7','8','e']:
    goal_positions[number] = get_location(goal, number)

def prob(state):
    """
    Define la función de probabilidad en base a la métrica Manhattan.
    
    Arguments
    ---------
    state : array
        Tabla (arreglo) que representa el estado actual del puzzle.
        
    Returns
    -------
    distance : int
        Probabilidad radial basada en métrica Manhattan.
    """
    #Acumula las distancias
    distance = 0
    #Computa la distancia sum(|x-y|)
    for number in ['1','2','3','4','5','6','7','8','e']:
        row_new, col_new = get_location(state, number)
        row_new_goal, col_new_goal = goal_positions[number]
        distance += abs(row_new - row_new_goal) + abs(col_new - col_new_goal)
        #Distancia de linea recta (euclideana)
        #distance += np.sqrt(abs(row_new - row_new_goal)**2 + abs(col_new - col_new_goal)**2)
    
    probability = np.exp(-distance)
    return probability

In [6]:
initial = [['1','e','2'],
           ['6','3','4'],
           ['7','5','8']]

almost = [['1','2','3'],
           ['4','5','6'],
           ['7','e','8']]

print(prob(initial))
print(prob(almost))

6.14421235332821e-06
0.1353352832366127


## Algoritmo de búsqueda por probabilidad

En este caso, tomamos un algoritmo similar a $A^*$ pero que toma la probabilidad como heurística. La probabilidad de un camino será el resultado del producto de las probabilidades desde el estado inicial hasta el estado actual:

$$p(n) = \prod_{i=1}^{n-1} p(n|n-i)$$

La cola de prioridad tomará el elemento más probable en cada transición.

#### Cola de prioridad

Definirmeos una cola de prioridad que tome como tope al elemento que máximiza la probabilidad de transición:

In [7]:
class PriorityQueue(object):
    """
    Clase de una cola de prioridad en los problemas de búsqueda.
    """
    def __init__(self,h=lambda x: 1):
        self.queue = []
        self.h = h
  
    def __str__(self):
        return ' '.join([str(q) for q in self.queue])
  
    def isEmpty(self):
        """
        Returns
        -------
        empty : bool
            Devuelve el valor True si el queue está vacío
        """
        return self.queue == []
  
    def push(self, element):
        """
        Arguments
        ---------
        element : 
            El elemento que se va agregar al queue
        """
        self.queue.append(element)
  
    def pop(self):
        """
        Elimina el elemento con más valor según un peso f
        y regresa el elemento correspondiente a este valor.
        """
        #Encuentra el elemento máximo en base al costo
        argmax = np.argmax([element.p for element in self.queue])
        #Guarda el elemento máximo
        item = self.queue[argmax]
        #Borra este elemento de la cola
        del self.queue[argmax]
    
        return item
    
    def top(self):
        """
        Devuelve el elemento con la función de costo más alta.
        A diferencia de pop, no elimina el elemento de la pila.
        """
        #Encuentra el elemento máximo en base al costo
        argmax = np.argmax([element.p for element in self.queue])
        #Guarda el elemento máximo
        item = self.queue[argmax]
        
        return item

#### Nodos

Los nodos del árbol de búsqueda guardarán:

* Los estados del problema del puzzle.
* El padre o nodo generador.
* La acción que llevó a este nodo.
* La función probabilidad desde el estado inicial hasta este nodo.

In [8]:
class Node(object):
    """
    Clase para crear nodos con sus atributos.
    """
    def __init__(self):
        self.state = None
        self.parent = None 
        self.action = None
        self.p = 1
        
    def __str__(self):
        return str(self.state)

#### Función de expansión de nodo

El nodo se expande según los estados que puedan resultar de las acciones posibles. En la expansión se genera los nodos hijos y en cada uno de estos se guarda la probabilidad de este nodo como el producto de las probabilidades desde el estado inicial hasta el actual.

In [9]:
def expand(problem, node):
    """
    Función para expandir los nodos dado el problema.
    
    Argumentos
    ---------
    problem : objeto
        Problema de búsqueda.
    node : objeto
        Nodo que se va a expandir.
        
    Salida
    ------
    childs :generator
        Generador con los hijos del nodo.
    """
    #Nodo en el que se inicia
    s = node.state
    for action in problem.actions():
        #Ejecuta la acción
        new_s = problem.result(action)
        
        #Genera un nuevo nodo
        new_node = Node()
        new_node.state = new_s
        new_node.parent = node 
        new_node.action = action 
        
        #Probabilidad del nodo
        new_node.p = node.p*prob(new_s)

        yield new_node

## Implementación del algoritmo

El algoritmo es similar a  $A^*$ o Best-First (Mejor primero), donde toma como función de costo a la función de probabilidad.

In [10]:
def ProbSearch(problem):
    """
    Algoritmo de búsqueda por probabilidad.
    
    Argumentos
    ----------
    problem : objeto
        Problema de búsquda.
        
    Salida
    ------
    nodes : list
        Lista de los nodos que llevan a la solución.
    """
    #Almacenamiento de nodos
    nodes = []
    #Nodo inicial
    node = Node()
    node.state = problem.initial
    #Frontera con cola de prioridad
    frontier = PriorityQueue()
    frontier.push(node)
    #Nodos alcanzados
    reached = {str(problem.initial):node}

    #Mientras la frontera no esté vacía
    while frontier.isEmpty() == False:
        #Pop en frontera
        node = frontier.pop()
        #Guarda el nodo en la lista
        nodes.append(node)
        #Actualiza el estado del problema
        problem.state = node.state

        #Si llega a un estado final se detiene 
        #y regresa el camino de nodos
        if problem.state == problem.goal:
            return nodes

        #Expande el nodo actual
        expantion = expand(problem, node)
        for child in expantion:
            #print(child.f)
            #Guarda estado de los hijos
            state = str(child.state)
            #Guarda los hijos no alcanzados o que tengan mayor probabilidad
            if state not in reached.keys() or child.p > reached[state].p:
                reached[state] = child
                frontier.push(child)

    #Si no logra llegar a un nodo final
    #El algoritmo regresa mensaje de error
    return "No se ha logrado llegar a un estado final."

##### Prueba del método

Ahora podemos probar el algoritmo en el problema de puzzle que hemos definido. Le daremos un estado inicial y dejaremos que $A^*$ encuentre las acciones necesarias para poder obtener la solución al puzzle, es decir, llegar al estado meta.

In [11]:
"""
#Estado más simple
initial = [['1', '2', '3'], 
           ['4', 'e', '5'], 
           ['7', '8', '6']]
"""
#"""
#Estado dicicultad intermedia
initial = [['1','e','2'],
           ['6','3','4'],
           ['7','5','8']]
#"""
"""
#Este inicial requiere más iteraciones
initial = [['5', '4', '2'], 
           ['3', '1', '7'], 
           ['e', '6', '8']]
           
"""

#Creamos el modelo del puzzle
puzzle = Puzzle(initial)

Aplicando el método podemos ver cuáles son las acciones para llegar al estado final.

In [12]:
#Aplicación de A*
result = ProbSearch(puzzle)

#Cuenta cuántos pasos para llegar a la solución
total_nodos = 0
for node in result:
    #Imprime acciones y estados
    #print("{}------>\n{}".format(node.action, np.asarray(node.state)))
    total_nodos += 1
    
print('Total de nodo en árbol: ', total_nodos)

Total de nodo en árbol:  790


### Visualización del camino hacia la meta

El camino que obtenemos a partir del árbol lo podemos obtener regresando en el árbol a partir de la hoja meta hasta la raíz del estado inicial. Recorriendo este camino obtenemos las acciones que llevan a la solución.

In [13]:
def get_path(search_tree, root):
    """
    Recorre el árbol de búsqueda de atrás hacia el inicio para obtener el camino de acciones
    que llevaron a la meta.
    """
    #Meta
    leaf = search_tree[-1]
    #Padre de la meta
    parent = leaf.parent
    #Adjunta la meta y su padre al camino
    path = [leaf, parent]
    #Recorre el árbol de búsqueda hasta el inicial
    while parent.state != root:
        parent = parent.parent
        #Adjunta los padres en el camino
        path.append(parent)
        
    return path[::-1]

pasos = 0
for n in get_path(result, initial):
    print("{}, prob: {}----->\n{}".format(n.action, n.p, np.asarray(n.state)))
    pasos += 1
    
print('Total de acciones: ', pasos)

None, prob: 1----->
[['1' 'e' '2']
 ['6' '3' '4']
 ['7' '5' '8']]
2, prob: 4.5399929762484854e-05----->
[['1' '2' 'e']
 ['6' '3' '4']
 ['7' '5' '8']]
4, prob: 2.061153622438558e-09----->
[['1' '2' '4']
 ['6' '3' 'e']
 ['7' '5' '8']]
3, prob: 9.357622968840175e-14----->
[['1' '2' '4']
 ['6' 'e' '3']
 ['7' '5' '8']]
6, prob: 4.24835425529159e-18----->
[['1' '2' '4']
 ['e' '6' '3']
 ['7' '5' '8']]
1, prob: 2.6102790696677053e-23----->
[['e' '2' '4']
 ['1' '6' '3']
 ['7' '5' '8']]
2, prob: 1.6038108905486382e-28----->
[['2' 'e' '4']
 ['1' '6' '3']
 ['7' '5' '8']]
4, prob: 7.281290178321646e-33----->
[['2' '4' 'e']
 ['1' '6' '3']
 ['7' '5' '8']]
3, prob: 2.4426007377405287e-36----->
[['2' '4' '3']
 ['1' '6' 'e']
 ['7' '5' '8']]
6, prob: 8.19401262399052e-40----->
[['2' '4' '3']
 ['1' 'e' '6']
 ['7' '5' '8']]
4, prob: 2.7487850079102163e-43----->
[['2' 'e' '3']
 ['1' '4' '6']
 ['7' '5' '8']]
2, prob: 9.22114642292588e-47----->
[['e' '2' '3']
 ['1' '4' '6']
 ['7' '5' '8']]
1, prob: 2.28569367